In [1]:
import tensorflow as tf
import math
from tensorflow.examples.tutorials.mnist import input_data
import numpy
#from tensorflow.examples.tutorials.mnist import mnist

# 1. Load data

In [2]:
mnist_data = input_data.read_data_sets("./datasets/mnist_data/", False) # one hot encoding must be false here
mnist_data.train.labels.shape

Extracting ./datasets/mnist_data/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist_data/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist_data/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist_data/t10k-labels-idx1-ubyte.gz


(55000,)

# 2. Model Parameters

In [3]:
batch_size = 100
image_classes = 10
image_size = 28
hidden1_units = 128
hidden2_units = 32
image_pixels= image_size * image_size
images = tf.placeholder(tf.float32, shape=(batch_size, image_pixels))
labels = tf.placeholder(tf.int32, shape=(batch_size))

# 3. Build the Graph

In [4]:
with tf.name_scope('hidden1'):
    weights = tf.Variable(
        tf.truncated_normal([image_pixels, hidden1_units], stddev = 1.0 / math.sqrt(float(image_pixels))), name='weights')
    biases = tf.Variable(tf.zeros([hidden1_units]), name='biases')
    hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
hidden1.shape

TensorShape([Dimension(100), Dimension(128)])

In [5]:
with tf.name_scope('hidden2'):
    weights = tf.Variable(
        tf.truncated_normal([hidden1_units, hidden2_units], stddev=1.0/math.sqrt(float(hidden1_units))), name='weights')
    biases = tf.Variable(tf.zeros([hidden2_units]), name='biases')
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
hidden2.shape

TensorShape([Dimension(100), Dimension(32)])

In [7]:
with tf.name_scope('softmax_linear'):
    weights = tf.Variable(
        tf.truncated_normal([hidden2_units, image_classes], stddev=1.0/math.sqrt(float(hidden2_units))), name='weights')
    biases = tf.Variable(tf.zeros([image_classes]), name='biases')
    logits = tf.matmul(hidden2, weights) + biases
print(logits.get_shape())

(100, 10)


# 4. Define Loss Function

In [8]:
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits, name='xentropy')
cross_entropy.shape
labels.shape

TensorShape([Dimension(100)])

In [9]:
loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
loss.shape

TensorShape([])

# 5. Training

In [10]:
tf.summary.scalar('loss', loss)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [11]:
learning_rate = 0.1
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

In [12]:
global_step = tf.Variable(0, name='global_step', trainable=False)
train_op = optimizer.minimize(loss, global_step=global_step)

In [13]:
# build evaluate metrix
correct_prediction = tf.nn.in_top_k(logits, labels, 1)
accuracy = tf.reduce_sum(tf.cast(correct_prediction, tf.float16))
tf.summary.scalar("accuracy", accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [15]:
with tf.Session() as sess:
    # build summary saver
    saver = tf.train.Saver()
    # Build the summary Tensor based on the TF collection of Summaries.
    summary_merged = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter("./logs/mnist/fully_connected", sess.graph)
    init = tf.global_variables_initializer()
    sess.run(init)
    for step in xrange(1000):
        batch_images, batch_labels = mnist_data.train.next_batch(100, False)
        feed_dict = {images: batch_images, labels: batch_labels}
        _, loss_value = sess.run([train_op, loss], feed_dict=feed_dict)
        if step % 100 == 0:
            step_accuracy = sess.run(accuracy, feed_dict=feed_dict)
            print("step: {}, accuracy: {}, loss_value: {}".format(step, step_accuracy, loss_value))
            # Update the events file.
            summary_str = sess.run(summary_merged, feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, step)
            print("add summary for step {}".format(step))

step: 0, accuracy: 5.0, loss_value: 2.33468699455
add summary for step 0
step: 100, accuracy: 84.0, loss_value: 0.56485170126
add summary for step 100
step: 200, accuracy: 93.0, loss_value: 0.393768996
add summary for step 200
step: 300, accuracy: 92.0, loss_value: 0.411037325859
add summary for step 300
step: 400, accuracy: 94.0, loss_value: 0.340746223927
add summary for step 400
step: 500, accuracy: 90.0, loss_value: 0.373371213675
add summary for step 500
step: 600, accuracy: 91.0, loss_value: 0.397236406803
add summary for step 600
step: 700, accuracy: 98.0, loss_value: 0.202075913548
add summary for step 700
step: 800, accuracy: 95.0, loss_value: 0.183002889156
add summary for step 800
step: 900, accuracy: 97.0, loss_value: 0.199020683765
add summary for step 900
